In [ ]:
from music21 import converter, instrument, note, chord, stream, midi
from google.colab import files
import glob
import os
import gzip
import tarfile    
from torchvision import datasets                  
import numpy as np
import torch
import torch.nn as nn
import torch.optim
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
from tqdm import tqdm

assert torch.cuda.is_available()

In [ ]:
# Import Drive for easier saving
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def download_data(filepath):
    if not os.path.exists(os.path.join(filepath, 'mozart_sonatas.tar.gz')):
      datasets.utils.download_url('https://github.com/Foundations-of-Applied-Mathematics/Data/raw/master/RNN/mozart_sonatas.tar.gz', filepath, 'mozart_sonatas.tar.gz', None)

    print('Extracting {}'.format('mozart_sonatas.tar.gz'))
    gzip_path = os.path.join(filepath, 'mozart_sonatas.tar.gz')
    with open(gzip_path.replace('.gz', ''), 'wb') as out_f, gzip.GzipFile(gzip_path) as zip_f:
      out_f.write(zip_f.read())

    print('Untarring {}'.format('mozart_sonatas.tar'))
    tar_path = os.path.join(filepath,'mozart_sonatas.tar')
    z = tarfile.TarFile(tar_path)
    z.extractall(tar_path.replace('.tar', ''))



**Problem 1**:

Download the data.
Write a function that accepts the path to the .mid files, parses the files, and returns a list of the $119348$ pitches as strings. 
For the Chords, join the pitches of the notes in the chords with a . as in D3.D2.

Print the length of your list and the number of unique pitches.


**Problem 2.** 

Using the list returned in Problem 1, create the training and testing DataLoaders.
Make sure to do all of the following steps:


1) Convert the pitches to integers.
2) Split the data into Long tensors of length 100.
3) Create the labels.
4) Randomly split the data into training and test sets using an 80/20 split.
5) Create the DataLoaders for both sets of data, using batch_size=128 for the training data and batch_size=1 for the test data.
 

Print the length of each DataLoader.


**Problem 3**

Create the network class.
Include at least $3$ LSTM layers, each followed by Dropout layers with probability $.3.$
Also have at least $2$ Linear layers.
The last LSTM layer and each of the Linear layers should be followed by a BatchNorm1d layer, for at least $3$ total batchnorm layers.
The final layer should be a Softmax activation.

Initialize the model, loss as CrossEntropyLoss, and optimizer as RMSprop.

Train the model until the accuracy is above 95\%. This can take?



After taking a backwards step during training, scale the gradients using 
nn.utils.clip_grad_norm_(model.parameters(), 5)
This will ensure that the gradients are reasonably sized so that the model can learn.

At the end of each epoch, calculate the accuracy and mean loss on the test data.
Remember to change the model to eval() mode when running the test data and train() when running on the training data.

After the accuracy is above 95\%, plot the training and test losses versus epochs on the same plot.
Plot the accuracy versus epochs.


In [ ]:
class RNN(nn.Module):
    '''
    Recurrent Neural Network Class
    '''
    def __init__(self,n_notes,embedding_dim):
      super(Network, self).__init__()
      # initialize layers
      
    def forward(self, x, hidden):
      # forward pass

    def initHidden(self,batch_size):
      # initialize the hidden layers
      weight = next(self.parameters()).data
      h0 = weight.new(self.num_layers, batch_size, self.hidden_size).zero_().to(device)
      h1 = weight.new(self.num_layers, batch_size, self.hidden_size).zero_().to(device)
      return (h0, h1)

**Problem 4.** 
Write a function that randomly chooses a sequence in the test data and predicts
the next 500 elements. 
Return a list of the 600 pitches in the sequence.

In [ ]:
def load_model(filename):
  """ Load a saved model to continue training or evaluate """
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  model = Network(n_vocab,32)
  model = model.to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.RMSprop(model.parameters(),lr=.001)

  checkpoint = torch.load(filename,map_location=torch.device('cpu'))
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  last_epoch = checkpoint['epoch']
  loss = checkpoint['loss']
  model.eval() # For evaluation only
  
  return model, criterion, optimizer

**Problem 5.** 
Convert the sequence from Problem 4 into note and chord objects and save it to
'mozart.mid'.